In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd

In [3]:
path_df = pd.read_csv('opendata110road.csv')

In [4]:
# 0只是欄首的中文翻譯,drop掉
path_df.drop(index=path_df.index[0], 
        axis=0, 
        inplace=True)

In [5]:
path_df.reset_index(drop=True)

,city,site_id,road
0,金門縣,金門縣金城鎮,中環路
1,金門縣,金門縣金城鎮,中興市場
2,金門縣,金門縣金城鎮,中興路
3,金門縣,金門縣金城鎮,伯玉路
4,金門縣,金門縣金城鎮,光前路
...,...,...,...
34455,桃園市,桃園市復興區,霞雲
34456,桃園市,桃園市復興區,霞雲坪
34457,桃園市,桃園市復興區,頭角
34458,桃園市,桃園市復興區,高坡


In [6]:
path_df.head()

,city,site_id,road
1,金門縣,金門縣金城鎮,中環路
2,金門縣,金門縣金城鎮,中興市場
3,金門縣,金門縣金城鎮,中興路
4,金門縣,金門縣金城鎮,伯玉路
5,金門縣,金門縣金城鎮,光前路


In [7]:
# 取區域，即site_id為後三碼

In [8]:
path_df['region'] = path_df['site_id'].str[-3:]

In [9]:
path_df.head()

,city,site_id,road,region
1,金門縣,金門縣金城鎮,中環路,金城鎮
2,金門縣,金門縣金城鎮,中興市場,金城鎮
3,金門縣,金門縣金城鎮,中興路,金城鎮
4,金門縣,金門縣金城鎮,伯玉路,金城鎮
5,金門縣,金門縣金城鎮,光前路,金城鎮


In [10]:
# pip install selenium+

In [11]:
import datetime
import logging
import os
import re
import sys
import time
import traceback
from datetime import datetime

import pandas as pd
import win32com.client as w3c
from bs4 import BeautifulSoup
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support.ui import WebDriverWait

In [12]:
current_version = "99"

In [13]:
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--start-maximized')
driver = webdriver.Chrome(f"chromedriver.exe")

C:\Users\willy\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [ ]:
driver.get("https://www.ris.gov.tw/app/portal/3053")

In [ ]:
# f* this...
wait(driver, 30).until(EC.frame_to_be_available_and_switch_to_it((By.ID,"content-frame")))

In [ ]:
house_number_button = driver.find_element(By.CSS_SELECTOR, "[data-type='doorplate']").click()

# 開始根據df 點擊地圖上的縣市

In [ ]:
idx = 30983 #桃園市桃園區三元街

In [ ]:
city = path_df['city'].iloc[idx]

In [ ]:
# path_df[path_df['city'].str.contains('桃園')]

In [ ]:
#不知為啥只能怎樣 反正要把字串符號加進去
city = f"""\"{city}\""""

In [ ]:
city

In [ ]:
driver.find_element(By.XPATH, (f"""//area[contains(@title, {city})]""")).click()

In [ ]:
time.sleep(2)

In [ ]:
region = path_df['region'].iloc[idx]
region = f"""{region}"""

In [ ]:
region_select = Select(driver.find_element(By.XPATH, (f"""//select[@id='areaCode']""")))

In [ ]:
region_select.select_by_visible_text(f"""{region}""")

In [ ]:
road = path_df['road'].iloc[idx]

In [ ]:
#寫路
driver.find_element(By.XPATH, (f"""//input[@name='street']""")).send_keys(road)

In [ ]:
# TODO :  驗證碼破解

#然後送出

In [ ]:
# wait(driver,5).until(EC.presence_of_element_located(
#     (By.XPATH, "//td[@id='next_result-pager' and @class = 'ui-pg-button ui-corner-all ui-state-disabled']"))
#                     )

In [ ]:
wait(driver,5).until(EC.element_to_be_clickable((By.XPATH, "//td[@id='next_result-pager']")))

In [ ]:
# # 確認next page按鈕為disable的布林函數 # not fixed
# def find_next_page_disabled():
#     try:
#         driver.find_element(By.XPATH, f"""//td[@id='next_result-pager' and @class='ui-pg-button ui-corner-all ui-state-disabled']""")
#         return True
#     except:
#         return False

In [ ]:
address_list = []
# 每五十個結果就要按下一頁，直到不能為止
# while wait(driver,5).until(EC.element_to_be_clickable((By.XPATH, "//td[@id='next_result-pager']"))):
while True:
    # 拿回所有門牌的結果
    result_address = driver.find_elements(By.XPATH, (f"""//td[@data-jqlabel='門牌資料']"""))
    
    for i in result_address:
        address_list.append(i.text)
        
    #拿到結果，按下一頁(超過五十個結果要額外按下一頁)
    try:
        driver.find_element(By.XPATH, f"""//td[@id='next_result-pager']""").click()
    except:
        #如果不能按下一頁(即最後一頁)則停止蒐資料
        break
    
    #等個2秒讓頁面讀取
    time.sleep(2)
    
#     #如果不能按下一頁(即最後一頁)則停止蒐資料 #這版本會有問題 沒用 直接用上面的方式吧
#     if find_next_page_disabled():
#         break

In [ ]:
len(address_list)

In [6]:
from geopy.geocoders import Nominatim

geolocation = Nominatim(user_agent="geotest")

location = geolocation.reverse("25.046517, 121.460614")
print(location.address)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=25.046517&lon=121.460614&format=json&addressdetails=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1108)')))

In [ ]:
def addr2GPS(address):
    